In [13]:
!nvidia-smi

Fri Mar 17 03:05:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:08:00.0 Off |                  N/A |
|  0%   31C    P0    71W / 250W |      1MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:09:00.0 Off |                  N/A |
|  0%   

In [1]:
import numpy as np

In [2]:
f_key_path = f'PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META_key.npy'

In [4]:
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'

In [5]:
f_key = np.load(f'{engram_dir}{f_key_path}').tolist()  

In [8]:
f_key

[b'above',
 b'according',
 b'account',
 b'acquire',
 b'acquired',
 b'acquisition',
 b'acquisitions',
 b'action',
 b'addition',
 b'additional',
 b'adds',
 b'administration',
 b'advantage',
 b'again',
 b'agency',
 b'agreed',
 b'agreement',
 b'airline',
 b'allow',
 b'almost',
 b'along',
 b'although',
 b'always',
 b'america',
 b'amount',
 b'analyst',
 b'announced',
 b'anything',
 b'apparently',
 b'approval',
 b'approved',
 b'april',
 b'areas',
 b'around',
 b'asked',
 b'asset',
 b'association',
 b'attorney',
 b'august',
 b'auto',
 b'away',
 b'bankers',
 b'banking',
 b'basis',
 b'become',
 b'began',
 b'behind',
 b'believe',
 b'below',
 b'best',
 b'bids',
 b'biggest',
 b'bill',
 b'bills',
 b'black',
 b'bond',
 b'book',
 b'bought',
 b'british',
 b'brokerage',
 b'brokers',
 b'budget',
 b'build',
 b'building',
 b'bush',
 b'businesses',
 b'buying',
 b'cable',
 b'california',
 b'call',
 b'called',
 b'calls',
 b'came',
 b'campaign',
 b"can't",
 b'canada',
 b'canadian',
 b'carry',
 b'cars',
 b'cases

In [9]:
import os
import sys
import numpy as np
import h5py
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from models.networks_2022 import BranchedNetwork
from data.ReconstructionTrainingDataset import CleanSoundsDataset
from data.ReconstructionTrainingDataset import NoisySoundsDataset
from data.ReconstructionTrainingDataset import GammaNoiseDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [2]:
pnet_name = 'pnet'
_train_datafile = 'clean_reconstruction_training_set'
SoundsDataset = CleanSoundsDataset
dset_kwargs = {}

In [16]:
snr = 'neg9'
bg = 'pink_noise'
pnet_name = 'pnet_snr-9_pinkNoise'
_train_datafile = 'hyperparameter_pooled_training_dataset_random_order_noNulls'
SoundsDataset = NoisySoundsDataset
dset_kwargs = {'snr': snr, 'bg': bg}

In [2]:
pnet_name = 'pnet_snr-9_pt2'
_train_datafile = 'hyperparameter_pooled_training_dataset_random_order_noNulls'
SoundsDataset = NoisySoundsDataset
dset_kwargs = {'snr': 'neg9'}

In [2]:
pnet_name = 'pnet_gammaNoise'
_train_datafile = 'gammaNoise_reconstruction_training_set'
SoundsDataset = GammaNoiseDataset
dset_kwargs = {}

In [3]:
from models.pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP

# Parameters

In [17]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 30
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 300

lr = 1E-5
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'
train_datafile = f'{engram_dir}{_train_datafile}.hdf5'

Device: cuda:0


In [18]:
!nvidia-smi

Mon Jan  2 13:21:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:8A:00.0 Off |                  N/A |
|  0%   20C    P8     7W / 250W |   3016MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [19]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/ctn/users/cf2794/Code/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/ctn/users/cf2794/Code/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [20]:
pnet = PNetClass(net, build_graph=True)

#### Load checkpoint if desired!

In [ ]:
pnet.load_state_dict(torch.load(
    f'{engram_dir}checkpoints/pnet/pnet-150-regular.pth'
    ))

In [ ]:
pnet.eval()

In [ ]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up train/test dataset

In [25]:
train_dataset = SoundsDataset(
    train_datafile, subset=.9, **dset_kwargs)
test_dataset = SoundsDataset(
    train_datafile, subset=.9,
    train = False, **dset_kwargs
    )

In [7]:
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [15]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

Training Epoch: 1 [30/36045]	Loss: 496.1280
Training Epoch: 1 [60/36045]	Loss: 538.0731
Training Epoch: 1 [90/36045]	Loss: 467.9078
Training Epoch: 1 [120/36045]	Loss: 498.6032
Training Epoch: 1 [150/36045]	Loss: 489.8661
Training Epoch: 1 [180/36045]	Loss: 473.2509
Training Epoch: 1 [210/36045]	Loss: 447.1912
Training Epoch: 1 [240/36045]	Loss: 572.2451
Training Epoch: 1 [270/36045]	Loss: 686.7305
Training Epoch: 1 [300/36045]	Loss: 640.4139
Training Epoch: 1 [330/36045]	Loss: 621.3936
Training Epoch: 1 [360/36045]	Loss: 586.4219
Training Epoch: 1 [390/36045]	Loss: 620.4456
Training Epoch: 1 [420/36045]	Loss: 577.4561
Training Epoch: 1 [450/36045]	Loss: 588.5890
Training Epoch: 1 [480/36045]	Loss: 526.9437
Training Epoch: 1 [510/36045]	Loss: 540.8100
Training Epoch: 1 [540/36045]	Loss: 535.0945
Training Epoch: 1 [570/36045]	Loss: 531.2734
Training Epoch: 1 [600/36045]	Loss: 523.5986
Training Epoch: 1 [630/36045]	Loss: 550.3634
Training Epoch: 1 [660/36045]	Loss: 539.7185
Training Epoc

Training Epoch: 1 [5400/36045]	Loss: 476.3601
Training Epoch: 1 [5430/36045]	Loss: 460.3575
Training Epoch: 1 [5460/36045]	Loss: 446.1279
Training Epoch: 1 [5490/36045]	Loss: 492.0411
Training Epoch: 1 [5520/36045]	Loss: 470.4036
Training Epoch: 1 [5550/36045]	Loss: 467.1649
Training Epoch: 1 [5580/36045]	Loss: 536.0987
Training Epoch: 1 [5610/36045]	Loss: 528.0709
Training Epoch: 1 [5640/36045]	Loss: 532.6179
Training Epoch: 1 [5670/36045]	Loss: 453.4260
Training Epoch: 1 [5700/36045]	Loss: 500.4660
Training Epoch: 1 [5730/36045]	Loss: 456.1789
Training Epoch: 1 [5760/36045]	Loss: 481.3965
Training Epoch: 1 [5790/36045]	Loss: 492.4836
Training Epoch: 1 [5820/36045]	Loss: 488.9148
Training Epoch: 1 [5850/36045]	Loss: 485.9364
Training Epoch: 1 [5880/36045]	Loss: 543.8331
Training Epoch: 1 [5910/36045]	Loss: 562.1533
Training Epoch: 1 [5940/36045]	Loss: 563.1474
Training Epoch: 1 [5970/36045]	Loss: 545.7425
Training Epoch: 1 [6000/36045]	Loss: 561.7643
Training Epoch: 1 [6030/36045]	Los

Training Epoch: 1 [10740/36045]	Loss: 580.1423
Training Epoch: 1 [10770/36045]	Loss: 550.4819
Training Epoch: 1 [10800/36045]	Loss: 523.7025
Training Epoch: 1 [10830/36045]	Loss: 566.7348
Training Epoch: 1 [10860/36045]	Loss: 569.9726
Training Epoch: 1 [10890/36045]	Loss: 583.8038
Training Epoch: 1 [10920/36045]	Loss: 498.6703
Training Epoch: 1 [10950/36045]	Loss: 415.2055
Training Epoch: 1 [10980/36045]	Loss: 396.3951
Training Epoch: 1 [11010/36045]	Loss: 461.2235
Training Epoch: 1 [11040/36045]	Loss: 461.7989
Training Epoch: 1 [11070/36045]	Loss: 457.2346
Training Epoch: 1 [11100/36045]	Loss: 455.7219
Training Epoch: 1 [11130/36045]	Loss: 499.8760
Training Epoch: 1 [11160/36045]	Loss: 514.7093
Training Epoch: 1 [11190/36045]	Loss: 561.9329
Training Epoch: 1 [11220/36045]	Loss: 534.0110
Training Epoch: 1 [11250/36045]	Loss: 538.6942
Training Epoch: 1 [11280/36045]	Loss: 534.0537
Training Epoch: 1 [11310/36045]	Loss: 538.8496
Training Epoch: 1 [11340/36045]	Loss: 511.6454
Training Epoc

Training Epoch: 1 [15990/36045]	Loss: 453.0525
Training Epoch: 1 [16020/36045]	Loss: 427.8663
Training Epoch: 1 [16050/36045]	Loss: 436.9943
Training Epoch: 1 [16080/36045]	Loss: 405.3495
Training Epoch: 1 [16110/36045]	Loss: 397.5897
Training Epoch: 1 [16140/36045]	Loss: 385.5065
Training Epoch: 1 [16170/36045]	Loss: 453.7315
Training Epoch: 1 [16200/36045]	Loss: 477.3825
Training Epoch: 1 [16230/36045]	Loss: 465.6151
Training Epoch: 1 [16260/36045]	Loss: 569.4929
Training Epoch: 1 [16290/36045]	Loss: 550.5336
Training Epoch: 1 [16320/36045]	Loss: 539.1475
Training Epoch: 1 [16350/36045]	Loss: 579.8715
Training Epoch: 1 [16380/36045]	Loss: 538.3758
Training Epoch: 1 [16410/36045]	Loss: 539.5995
Training Epoch: 1 [16440/36045]	Loss: 527.6057
Training Epoch: 1 [16470/36045]	Loss: 528.7730
Training Epoch: 1 [16500/36045]	Loss: 518.8847
Training Epoch: 1 [16530/36045]	Loss: 485.7346
Training Epoch: 1 [16560/36045]	Loss: 498.8284
Training Epoch: 1 [16590/36045]	Loss: 518.0984
Training Epoc

Training Epoch: 1 [21240/36045]	Loss: 430.5455
Training Epoch: 1 [21270/36045]	Loss: 468.0368
Training Epoch: 1 [21300/36045]	Loss: 504.3957
Training Epoch: 1 [21330/36045]	Loss: 490.6345
Training Epoch: 1 [21360/36045]	Loss: 498.6601
Training Epoch: 1 [21390/36045]	Loss: 500.2559
Training Epoch: 1 [21420/36045]	Loss: 503.0526
Training Epoch: 1 [21450/36045]	Loss: 496.1414
Training Epoch: 1 [21480/36045]	Loss: 495.3603
Training Epoch: 1 [21510/36045]	Loss: 542.2305
Training Epoch: 1 [21540/36045]	Loss: 596.7069
Training Epoch: 1 [21570/36045]	Loss: 614.3982
Training Epoch: 1 [21600/36045]	Loss: 591.5840
Training Epoch: 1 [21630/36045]	Loss: 623.2941
Training Epoch: 1 [21660/36045]	Loss: 590.6331
Training Epoch: 1 [21690/36045]	Loss: 637.8439
Training Epoch: 1 [21720/36045]	Loss: 567.3512
Training Epoch: 1 [21750/36045]	Loss: 601.4503
Training Epoch: 1 [21780/36045]	Loss: 421.8167
Training Epoch: 1 [21810/36045]	Loss: 423.8259
Training Epoch: 1 [21840/36045]	Loss: 421.2581
Training Epoc

...

Training Epoch: 2 [11610/36045]	Loss: 579.9512
Training Epoch: 2 [11640/36045]	Loss: 578.9963
Training Epoch: 2 [11670/36045]	Loss: 601.8132
Training Epoch: 2 [11700/36045]	Loss: 556.0808
Training Epoch: 2 [11730/36045]	Loss: 591.2645
Training Epoch: 2 [11760/36045]	Loss: 609.1748
Training Epoch: 2 [11790/36045]	Loss: 635.3881
Training Epoch: 2 [11820/36045]	Loss: 640.8660
Training Epoch: 2 [11850/36045]	Loss: 731.0140
Training Epoch: 2 [11880/36045]	Loss: 854.4921
Training Epoch: 2 [11910/36045]	Loss: 948.9555
Training Epoch: 2 [11940/36045]	Loss: 904.4485
Training Epoch: 2 [11970/36045]	Loss: 909.9544
Training Epoch: 2 [12000/36045]	Loss: 936.8904
Training Epoch: 2 [12030/36045]	Loss: 864.3895
Training Epoch: 2 [12060/36045]	Loss: 961.3657
Training Epoch: 2 [12090/36045]	Loss: 413.2813
Training Epoch: 2 [12120/36045]	Loss: 403.7952
Training Epoch: 2 [12150/36045]	Loss: 387.8820
Training Epoch: 2 [12180/36045]	Loss: 384.6813
Training Epoch: 2 [12210/36045]	Loss: 397.0682
Training Epoc

Training Epoch: 2 [16860/36045]	Loss: 495.9269
Training Epoch: 2 [16890/36045]	Loss: 471.3844
Training Epoch: 2 [16920/36045]	Loss: 531.3173
Training Epoch: 2 [16950/36045]	Loss: 510.6346
Training Epoch: 2 [16980/36045]	Loss: 510.8799
Training Epoch: 2 [17010/36045]	Loss: 496.2642
Training Epoch: 2 [17040/36045]	Loss: 514.5751
Training Epoch: 2 [17070/36045]	Loss: 525.8785
Training Epoch: 2 [17100/36045]	Loss: 516.7512
Training Epoch: 2 [17130/36045]	Loss: 460.2379
Training Epoch: 2 [17160/36045]	Loss: 414.4893
Training Epoch: 2 [17190/36045]	Loss: 394.1818
Training Epoch: 2 [17220/36045]	Loss: 426.6390
Training Epoch: 2 [17250/36045]	Loss: 433.8892
Training Epoch: 2 [17280/36045]	Loss: 460.5475
Training Epoch: 2 [17310/36045]	Loss: 430.2480
Training Epoch: 2 [17340/36045]	Loss: 452.4840
Training Epoch: 2 [17370/36045]	Loss: 453.9222
Training Epoch: 2 [17400/36045]	Loss: 463.7976
Training Epoch: 2 [17430/36045]	Loss: 480.9719
Training Epoch: 2 [17460/36045]	Loss: 477.5343
Training Epoc

Training Epoch: 2 [22110/36045]	Loss: 425.4095
Training Epoch: 2 [22140/36045]	Loss: 422.8157
Training Epoch: 2 [22170/36045]	Loss: 421.8476
Training Epoch: 2 [22200/36045]	Loss: 444.7056
Training Epoch: 2 [22230/36045]	Loss: 446.6365
Training Epoch: 2 [22260/36045]	Loss: 432.9251
Training Epoch: 2 [22290/36045]	Loss: 479.9900
Training Epoch: 2 [22320/36045]	Loss: 525.6042
Training Epoch: 2 [22350/36045]	Loss: 525.9236
Training Epoch: 2 [22380/36045]	Loss: 528.0861
Training Epoch: 2 [22410/36045]	Loss: 512.3497
Training Epoch: 2 [22440/36045]	Loss: 506.3732
Training Epoch: 2 [22470/36045]	Loss: 525.3086
Training Epoch: 2 [22500/36045]	Loss: 501.8947
Training Epoch: 2 [22530/36045]	Loss: 537.1650
Training Epoch: 2 [22560/36045]	Loss: 529.3901
Training Epoch: 2 [22590/36045]	Loss: 560.5935
Training Epoch: 2 [22620/36045]	Loss: 598.1628
Training Epoch: 2 [22650/36045]	Loss: 603.1258
Training Epoch: 2 [22680/36045]	Loss: 609.3254
Training Epoch: 2 [22710/36045]	Loss: 624.5020
Training Epoc

Training Epoch: 2 [27360/36045]	Loss: 373.0784
Training Epoch: 2 [27390/36045]	Loss: 374.4119
Training Epoch: 2 [27420/36045]	Loss: 410.9880
Training Epoch: 2 [27450/36045]	Loss: 505.6653
Training Epoch: 2 [27480/36045]	Loss: 506.7975
Training Epoch: 2 [27510/36045]	Loss: 515.5586
Training Epoch: 2 [27540/36045]	Loss: 501.0588
Training Epoch: 2 [27570/36045]	Loss: 524.7844
Training Epoch: 2 [27600/36045]	Loss: 507.8185
Training Epoch: 2 [27630/36045]	Loss: 512.8701
Training Epoch: 2 [27660/36045]	Loss: 511.4330
Training Epoch: 2 [27690/36045]	Loss: 515.8909
Training Epoch: 2 [27720/36045]	Loss: 554.6657
Training Epoch: 2 [27750/36045]	Loss: 529.4340
Training Epoch: 2 [27780/36045]	Loss: 524.7893
Training Epoch: 2 [27810/36045]	Loss: 524.4899
Training Epoch: 2 [27840/36045]	Loss: 519.1077
Training Epoch: 2 [27870/36045]	Loss: 528.2435
Training Epoch: 2 [27900/36045]	Loss: 452.8544
Training Epoch: 2 [27930/36045]	Loss: 428.9923
Training Epoch: 2 [27960/36045]	Loss: 398.4507
Training Epoc

Training Epoch: 2 [32610/36045]	Loss: 376.7801
Training Epoch: 2 [32640/36045]	Loss: 510.9744
Training Epoch: 2 [32670/36045]	Loss: 547.1160
Training Epoch: 2 [32700/36045]	Loss: 542.2655
Training Epoch: 2 [32730/36045]	Loss: 502.3091
Training Epoch: 2 [32760/36045]	Loss: 530.9139
Training Epoch: 2 [32790/36045]	Loss: 528.4348
Training Epoch: 2 [32820/36045]	Loss: 523.3292
Training Epoch: 2 [32850/36045]	Loss: 464.5369
Training Epoch: 2 [32880/36045]	Loss: 386.2833
Training Epoch: 2 [32910/36045]	Loss: 389.9991
Training Epoch: 2 [32940/36045]	Loss: 394.7760
Training Epoch: 2 [32970/36045]	Loss: 393.2508
Training Epoch: 2 [33000/36045]	Loss: 399.3047
Training Epoch: 2 [33030/36045]	Loss: 386.0124
Training Epoch: 2 [33060/36045]	Loss: 370.6916
Training Epoch: 2 [33090/36045]	Loss: 382.9461
Training Epoch: 2 [33120/36045]	Loss: 619.6291
Training Epoch: 2 [33150/36045]	Loss: 592.3224
Training Epoch: 2 [33180/36045]	Loss: 569.4686
Training Epoch: 2 [33210/36045]	Loss: 603.7356
Training Epoc